<a href="https://colab.research.google.com/github/ridwanbello/accent_recognition/blob/main/Evaluation_of_different_Accents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets==2.12.0
!pip install fsspec==2023.9.2

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2023.9.2 which is incompatible.


In [4]:
import os
import itertools
import pandas as pd
import soundfile as sf
from datasets import load_dataset

base_path = "/content/afrispeech"
main_path = os.path.join(base_path, "yoruba")
os.makedirs(main_path, exist_ok=True)

ds = load_dataset("tobiolatunji/afrispeech-200", "yoruba", split="train", streaming=True, cache_dir=None)

data_records = []

for i, example in enumerate(itertools.islice(ds, 0, 100)):
    audio_array = example["audio"]["array"]
    sampling_rate = example["audio"]["sampling_rate"]
    accent = example["accent"]
    reference_text = example["transcript"]
    file_path = f"{main_path}/{accent}_{i}.wav"
    sf.write(file_path, audio_array, sampling_rate)

    data_records.append({
        "audio_path": file_path,
        "reference_text": reference_text
    })

# Convert to DataFrame and save CSV
df = pd.DataFrame(data_records)
csv_path = os.path.join(base_path, "yoruba_audio_text.csv")
df.to_csv(csv_path, index=False)
print(f"CSV saved to {csv_path}")

Reading metadata...: 14369it [00:00, 23206.01it/s]


CSV saved to /content/afrispeech/yoruba_audio_text.csv


In [ ]:
def download_accent_csv()

In [1]:
pip install faster-whisper jiwer pandas soundfile tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 128.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00


In [9]:
import os, time
import pandas as pd
import soundfile as sf
from tqdm import tqdm
from faster_whisper import WhisperModel
import jiwer

In [2]:
import soundfile as sf
import itertools
from datasets import load_dataset
import os
from typing import List, Tuple
from pathlib import Path

In [5]:
CSV_PATH = "/content/afrispeech/yoruba_audio_text.csv"

MODELS = [
    ("small",   {"compute_type": "float16"}),  # set int8 on CPU if needed
    ("medium",  {"compute_type": "float16"}),
    ("large-v3",{"compute_type": "float16"}),
]

df = pd.read_csv(CSV_PATH)

In [6]:
def audio_duration(path):
    info = sf.info(path)
    return info.frames / float(info.samplerate)

In [7]:
def evaluate_model(model_name, model_kwargs):
    model = WhisperModel(model_name, **model_kwargs)
    rows = []
    total_ref, total_hyp = [], []
    total_audio_sec, total_decode_sec = 0.0, 0.0

    for _, r in tqdm(df.iterrows(), total=len(df)):
        wav = r["audio_path"]
        ref = str(r["reference_text"])
        lang = r["language"] if "language" in r and pd.notna(r["language"]) else None

        t0 = time.time()
        # language can be None (auto-detect) or e.g., "sw", "yo", "ha", "am", "ig", etc.
        segments, info = model.transcribe(wav, language=lang)
        hyp_text = "".join([s.text for s in segments]).strip()
        t1 = time.time()

        dur = audio_duration(wav)
        dec = t1 - t0
        total_audio_sec += dur
        total_decode_sec += dec

        rows.append({
            "model": model_name,
            "audio_path": wav,
            "reference": ref,
            "hypothesis": hyp_text,
            "duration_sec": dur,
            "decode_sec": dec
        })
        total_ref.append(ref)
        total_hyp.append(hyp_text)

    # Metrics
    wer = jiwer.wer(total_ref, total_hyp)
    cer = jiwer.cer(total_ref, total_hyp)
    rtf = total_decode_sec / max(total_audio_sec, 1e-6)

    agg = {"model": model_name, "WER": wer, "CER": cer, "RTF": rtf,
           "total_audio_sec": total_audio_sec, "total_decode_sec": total_decode_sec}
    return pd.DataFrame(rows), agg

In [10]:
all_rows = []
aggs = []
for name, kwargs in MODELS:
    per_utts, agg = evaluate_model(name, kwargs)
    all_rows.append(per_utts)
    aggs.append(agg)

per_utterance = pd.concat(all_rows, ignore_index=True)
summary = pd.DataFrame(aggs).sort_values("WER")

per_utterance.to_csv("whisper_eval_per_utterance.csv", index=False)
summary.to_csv("whisper_eval_summary.csv", index=False)
print(summary)

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

100%|██████████| 100/100 [00:45<00:00,  2.22it/s]


...

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

100%|██████████| 100/100 [01:00<00:00,  1.64it/s]

      model       WER       CER       RTF  total_audio_sec  total_decode_sec
2  large-v3  0.383333  0.165109  0.076537       793.480952         60.730721
1    medium  0.402778  0.184400  0.089263       793.480952         70.828783
0     small  0.460417  0.211958  0.056576       793.480952         44.891630


In [11]:
import os
import itertools
import pandas as pd
import soundfile as sf
from tqdm import tqdm
from faster_whisper import WhisperModel
import jiwer
from math import ceil

# Input CSV columns: audio_path, reference_text
CSV_PATH = "/content/afrispeech/yoruba_audio_text.csv"

MODELS = [
    ("tiny", {"compute_type": "float16"}),
    ("base", {"compute_type": "float16"}),
    ("small", {"compute_type": "float16"}),
    ("medium", {"compute_type": "float16"}),
    ("large-v2", {"compute_type": "float16"}),
    ("large-v3", {"compute_type": "float16"})
]

df = pd.read_csv(CSV_PATH)

def audio_duration(path):
    info = sf.info(path)
    return info.frames / float(info.samplerate)

def evaluate_model(model_name, model_kwargs, language=None):
    model = WhisperModel(model_name, **model_kwargs)
    rows = []
    total_ref, total_hyp = [], []
    total_audio_sec, total_decode_sec = 0.0, 0.0

    for _, r in tqdm(df.iterrows(), total=len(df)):
        wav = r["audio_path"]
        ref = str(r["reference_text"])

        t0 = time.time()
        segments, info = model.transcribe(wav, language=language)
        hyp_text = "".join([s.text for s in segments]).strip()
        t1 = time.time()

        dur = audio_duration(wav)
        dec = t1 - t0

        # Convert seconds to minutes and round
        dur_min = round(dur / 60, 2)
        dec_min = round(dec / 60, 2)

        total_audio_sec += dur
        total_decode_sec += dec

        rows.append({
            "model": model_name,
            "audio_path": wav,
            "reference": ref,
            "hypothesis": hyp_text,
            "duration_min": dur_min,
            "decode_min": dec_min
        })
        total_ref.append(ref)
        total_hyp.append(hyp_text)

    # Metrics
    wer = jiwer.wer(total_ref, total_hyp)
    cer = jiwer.cer(total_ref, total_hyp)
    rtf = round(total_decode_sec / max(total_audio_sec, 1e-6), 2)

    agg = {
        "model": model_name,
        "language": str(language),
        "WER": round(wer, 4),
        "CER": round(cer, 4),
        "RTF": rtf,
        "total_audio_minutes": round(total_audio_sec / 60, 2),
        "total_decode_minutes": round(total_decode_sec / 60, 2)
    }
    return pd.DataFrame(rows), agg

all_rows = []
aggs = []

for name, kwargs in MODELS:
    for lang in ["en", None]:
        print(f"Now working on {name} model size and {lang} lang")
        per_utts, agg = evaluate_model(name, kwargs, language=lang)
        all_rows.append(per_utts)
        aggs.append(agg)

# Concatenate all results
per_utterance_df = pd.concat(all_rows, ignore_index=True)
summary_df = pd.DataFrame(aggs).sort_values("WER")

# Save results
per_utterance_df.to_csv("whisper_eval_optimized_per_utterance.csv", index=False)
summary_df.to_csv("whisper_eval_optimized_summary.csv", index=False)

print(summary_df)


tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

100%|██████████| 100/100 [00:39<00:00,  2.52it/s]


...

vocabulary.txt: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

100%|██████████| 100/100 [01:01<00:00,  1.63it/s]

       model language     WER     CER   RTF  total_audio_minutes  \
8   large-v2       en  0.3757  0.1631  0.07                13.22   
9   large-v2     None  0.3764  0.1634  0.09                13.22   
11  large-v3     None  0.3826  0.1650  0.08                13.22   
10  large-v3       en  0.3840  0.1667  0.07                13.22   
6     medium       en  0.3882  0.1710  0.06                13.22   
7     medium     None  0.4062  0.1837  0.09                13.22   
4      small       en  0.4569  0.2114  0.05                13.22   
5      small     None  0.4604  0.2120  0.05                13.22   
2       base       en  0.5569  0.2749  0.05                13.22   
3       base     None  0.5611  0.2771  0.05                13.22   
1       tiny     None  0.6188  0.3335  0.05                13.22   
0       tiny       en  0.6222  0.3339  0.05                13.22   

    total_decode_minutes  
8                   0.93  
9                   1.15  
11                  1.02  
10     

In [12]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to save the CSV files
drive_path = "/content/drive/MyDrive/Afrispeech/whisper_evaluation_results/"

# Create directory if it doesn't exist
import os
os.makedirs(drive_path, exist_ok=True)

# Save CSV files to Google Drive
per_utterance_df.to_csv(os.path.join(drive_path, "whisper_eval_optimized_per_utterance.csv"), index=False)
summary_df.to_csv(os.path.join(drive_path, "whisper_eval_optimized_summary.csv"), index=False)

print(f"CSV files saved to {drive_path}")


Mounted at /content/drive
CSV files saved to /content/drive/MyDrive/Afrispeech/whisper_evaluation_results/


In [13]:
! pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-fxkhgiua
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-fxkhgiua
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=b56565099e6f19d35add3f0904aef932918d58187a64f0b4ccb5101875e59dab
  Stored in directory: /tmp/pip-ephem-wheel-cache-115teddr/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper


In [17]:
import os
import pandas as pd
import soundfile as sf
import time
from tqdm import tqdm
from faster_whisper import WhisperModel
import jiwer
from whisper.normalizers import EnglishTextNormalizer

MODELS = [
    ("tiny", {"compute_type": "float16"}),
    ("base", {"compute_type": "float16"}),
    ("small", {"compute_type": "float16"}),
    ("medium", {"compute_type": "float16"}),
    ("large-v2", {"compute_type": "float16"}),
    ("large-v3", {"compute_type": "float16"}),
]

normalizer = EnglishTextNormalizer()

def audio_duration(path):
    info = sf.info(path)
    return info.frames / float(info.samplerate)

def evaluate_whisper_csv(csv_path, output_dir="results"):
    os.makedirs(output_dir, exist_ok=True)
    df = pd.read_csv(csv_path)

    def evaluate_model(model_name, model_kwargs, language=None):
        model = WhisperModel(model_name, **model_kwargs)
        rows = []
        total_ref, total_hyp = [], []
        total_audio_sec, total_decode_sec = 0.0, 0.0

        for _, r in tqdm(df.iterrows(), total=len(df)):
            wav = r["audio_path"]
            ref = str(r["reference_text"])
            ref_norm = normalizer(ref)

            t0 = time.time()
            segments, info = model.transcribe(wav, language=language)
            hyp_text = "".join([s.text for s in segments]).strip()
            hyp_norm = normalizer(hyp_text)
            t1 = time.time()

            dur = audio_duration(wav)
            dec = t1 - t0

            dur_min = round(dur / 60, 2)
            dec_min = round(dec / 60, 2)

            rows.append({
                "model": model_name,
                "audio_path": wav,
                "reference": ref_norm,
                "hypothesis": hyp_norm,
                "duration_min": dur_min,
                "decode_min": dec_min
            })

            total_audio_sec += dur
            total_decode_sec += dec
            total_ref.append(ref_norm)
            total_hyp.append(hyp_norm)

        wer = jiwer.wer(total_ref, total_hyp)
        cer = jiwer.cer(total_ref, total_hyp)
        rtf = round(total_decode_sec / max(total_audio_sec, 1e-6), 2)

        lang_label = language if language is not None else "All"

        agg = {
            "model": model_name,
            "language": lang_label,
            "WER": round(wer, 4),
            "CER": round(cer, 4),
            "RTF": rtf,
            "total_audio_minutes": round(total_audio_sec / 60, 2),
            "total_decode_minutes": round(total_decode_sec / 60, 2)
        }
        return pd.DataFrame(rows), agg

    all_rows = []
    aggs = []

    for name, kwargs in MODELS:
        for lang in ["en", None]:
            print(f"Now working on {name} model size and {lang} lang")
            per_utts, agg = evaluate_model(name, kwargs, language=lang)
            all_rows.append(per_utts)
            aggs.append(agg)

    per_utterance_df = pd.concat(all_rows, ignore_index=True)
    summary_df = pd.DataFrame(aggs).sort_values("WER")

    per_utterance_csv = os.path.join(output_dir, f"{os.path.basename(csv_path).replace('.csv','')}_per_utterance.csv")
    summary_csv = os.path.join(output_dir, f"{os.path.basename(csv_path).replace('.csv','')}_summary.csv")

    per_utterance_df.to_csv(per_utterance_csv, index=False)
    summary_df.to_csv(summary_csv, index=False)

    print(f"Saved per-utterance results to {per_utterance_csv}")
    print(f"Saved summary results to {summary_csv}")

    return per_utterance_df, summary_df


In [ ]:
accent_csvs = ["yoruba_audio_text.csv"]
for csv in accent_csvs:
    print(f"Processing {csv}")
    csv_path = f"/content/afrispeech/{csv}"
    evaluate_whisper_csv(csv_path, output_dir="whisper_results")


Processing yoruba_audio_text.csv
Now working on tiny model size and en lang


100%|██████████| 100/100 [00:38<00:00,  2.60it/s]


Now working on tiny model size and None lang


100%|██████████| 100/100 [00:40<00:00,  2.45it/s]


Now working on base model size and en lang


...

100%|██████████| 100/100 [00:41<00:00,  2.43it/s]


Now working on small model size and None lang


100%|██████████| 100/100 [00:41<00:00,  2.40it/s]


Now working on medium model size and en lang


 15%|█▌        | 15/100 [00:07<00:38,  2.20it/s]